In [2]:
import pandas as pd
from collections import Counter
from profidaten_patstat_matching import Patstat, Profidaten, BestMatches, AusführendeStelle, load_clean_names_dict
from preprocessing import clean_company_names, SimpleGMBHRemover
from postprocessing import load_matching_results, classify_matching_results


# Analyzing the Patstat Subset

In [ ]:
print(f"There are a total of {len(Patstat.data):,} entries in the Patstat subset.")
print(f"There are {len(Patstat.data['person_name'].dropna()):,} entries with person_names in the Patstat subset.")
print(f"There are {len(Patstat.data['person_name'].dropna().drop_duplicates()):,} individual person_names in the Patstat subset.")


In [ ]:
# Cleaning the Patstat subset
output_file = "./data/clean_patstat_names.txt" 

names = Patstat.data["person_name"].dropna().drop_duplicates().tolist()
clean_company_names(names, output_file=output_file)


In [ ]:
#Individual Patstat person_names after cleaning the names
file = "./data/clean_patstat_names.txt" 
clean_names_dict = load_clean_names_dict(file)
print(f"There are {len(set(clean_names_dict.values())):,} individual Person Names in the Patstat dataset after cleaning the names.")


# Analyzing the Profi Dataset

In [ ]:
print(f"There are a total of {len(Profidaten.data):,} entries in the Profi extract.")
print(f"There are {len(AusführendeStelle.data):,} entries in the 'AusführendeStelle' column of the profi dataset after cleaning and removing duplicates.")

# Sting Matching the Patstat and the Profi Names. 

In [ ]:
file = "./data/clean_patstat_names.txt" 
clean_names_dict = load_clean_names_dict(file)
names = list(set(clean_names_dict.values()))

output_file = "./data/matching_results.txt" 
with open(output_file, "w") as file:
    file.write("Ausführende Stelle;Patstat\n")
BestMatches.find_closest_matches(names, out_file=output_file)

# Classifing the Matching Results using gpt-4o mini


In [3]:
import os

os.environ["OPENAI_API_KEY"] = "" # Set with your key
os.environ["MODEL"] = "gpt-4o-mini"

In [5]:
matching_results = load_matching_results("./data/matching_results.txt")
with open("data/matching_results_classified.txt", "w", encoding="utf-8") as file:
    file.write("Ausführende Stelle;Patstat;Classification\n")
    for i, (name0, name1) in  enumerate(zip(matching_results["Ausführende Stelle"], matching_results["Patstat"])):
                print(f"{i}/{len(matching_results)}", end="\r")
                classification_result = classify_matching_results(name0, name1)
                file.write(f"{name0};{name1};{classification_result}\n")
    

In [25]:
# Checking accuracy of string matching

classified_matching_results = pd.read_csv("data/matching_results_classified.txt", delimiter=";", lineterminator="\n")
predictions = classified_matching_results["Classification"]
C = Counter(predictions)
print(f"The ChatGPT classification showed a {C[True] *100/(C[False] + C[True]):.2f} % string matching accuracy.")


The ChatGPT classification showed a 35.00 % string matching accuracy.


In [14]:
# Creating a translation table for Ausführende Stelle to Patstat
classified_matching_results = pd.read_csv("data/matching_results_classified.txt", delimiter=";", lineterminator="\n")
translation_dict = {row["Ausführende Stelle"]: row["Patstat"] for i, row in classified_matching_results.iterrows() if row["Classification"]}

# Creating a reverse translation table from clean patstat name to normal Patstat name.
clean_patstat_names = pd.read_csv("data/clean_patstat_names.txt", delimiter=";", lineterminator="\n")
reverse_patstat_dict = {row[" Clean Name"]: row["Original Name"] for i, row in clean_patstat_names.iterrows()}

a = []
for i, x in enumerate(reverse_patstat_dict.keys()):
    a.append(x)
    if i == 100:
        break

def translate(name: str):
    name = SimpleGMBHRemover.preprocess_name(name)

    if name in translation_dict.keys():
        name = translation_dict[name]

        # Removing new line statement at the end
        if name[-2:] == "\\n": 
            name = name[:-2]

        name = reverse_patstat_dict[name] 
        return name
    else: 
        return "No Match Found!"

id_ausführende_stelle = Profidaten.data[["ID", "p_ausführendeStelle"]]
id_ausführende_stelle["Patstat Name"] = id_ausführende_stelle["p_ausführendeStelle"].apply(translate)
#id_ausführende_stelle = id_ausführende_stelle[id_ausführende_stelle["Patstat Name"] != "No Match Found!"]

# Adding the person_ids form patstat
person_ids = []
patstat_persons = Patstat.data.drop_duplicates(subset="person_id")

def extract_person_id(name:str):
    if name == "No Match Found!":
        return name
    patstat_with_name = patstat_persons[patstat_persons["person_name"] == name]
    person_ids = patstat_with_name["person_id"]
    return person_ids.to_list()
    
for i, name in enumerate(id_ausführende_stelle["Patstat Name"]):
    print(f"{i}/{len(id_ausführende_stelle)}", end='\r')
    person_id_candidates = extract_person_id(name)
    person_ids.append(person_id_candidates)

id_ausführende_stelle["Patstat person_id"] = person_ids

id_ausführende_stelle.to_excel("data/profi_patstat_translation.xlsx")






/tmp/ipykernel_771407/562216038.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_ausführende_stelle["Patstat Name"] = id_ausführende_stelle["p_ausführendeStelle"].apply(translate)


/tmp/ipykernel_771407/562216038.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_ausführende_stelle["Patstat person_id"] = person_ids
